In [2]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# USPTO - Patent ID's

In [ ]:
g_cpc_uspto = pd.read_csv('/mnt/hdd01/patentsview/Raw files/Raw csv files/g_cpc_current.csv')

In [ ]:
# Aggregate all the CPC codes for each patent
g_cpc_uspto_agg = g_cpc_uspto.groupby('patent_id')['cpc_class'].apply(list).reset_index()

In [ ]:
# Randomly sample 600.000 patents from the aggregated dataset, where 'cpc_class' does not contain "Y02"
g_cpc_uspto_agg_non_cleantech = g_cpc_uspto_agg[~g_cpc_uspto_agg['cpc_class'].progress_apply(lambda x: any('Y02' in s for s in x))]
g_cpc_uspto_agg_non_cleantech_sample = g_cpc_uspto_agg_non_cleantech.sample(n=600000, random_state=42)

In [ ]:
g_cpc_uspto_agg_non_cleantech_sample.to_csv('/mnt/hdd01/patentsview/Non Cleantech Patents - Classifier Set/g_uspto_non_cleantech_ids.csv', index=False)

# Reliance on Science - oaid's (USPTO + EPO)

In [2]:
df_oaid_uspto = pd.read_csv('/mnt/hdd01/patentsview/Reliance on Science - Cleantech Patents/df_oaid_Cleantech_Y02.csv', usecols=['oaid', 'patent_id'], dtype={'oaid': str, 'patent_id': str})
df_oaid_epo = pd.read_json('/mnt/hdd01/PATSTAT Working Directory/Reliance on Science/cleantech_epo_rel_on_science_abstract.json', dtype={'oaid': str, 'publn_nr': str})
# Drop all columns in df_oaid_epo except 'oaid' and 'publn_nr'
df_oaid_epo = df_oaid_epo[['oaid', 'publn_nr']]
# Delete everything after the '.' in oaid in df_oaid_uspto
df_oaid_uspto['oaid'] = df_oaid_uspto['oaid'].str.split('.').str[0]
# Merge df_oaid_uspto and df_oaid_epo on 'oaid'
df_oaid_uspto_epo = pd.merge(df_oaid_uspto, df_oaid_epo, on='oaid', how='outer')

In [3]:
df_pcs = pd.read_csv('/mnt/hdd01/Reliance on Science/Raw Files/_pcs_oa.csv', dtype={'oaid': str, 'patent_id': str})
# Extract everything before first hypen in column 'patent' into new column 'origin'
df_pcs['origin'] = df_pcs['patent'].str.split('-').str[0]
# Limit df_pcs to only patents with origin 'us' or 'ep'
df_pcs = df_pcs[(df_pcs['origin'] == 'us') | (df_pcs['origin'] == 'ep')]
# Aggregate df_pcs on oaid and list all patent
df_pcs_agg = df_pcs.groupby('oaid')['patent'].apply(list).reset_index()

In [12]:
# Randomly sample 800.000 patents from df_pcs where 'oaid' is not in df_oaid_uspto_epo
df_pcs_non_cleantech = df_pcs_agg[~df_pcs_agg['oaid'].isin(df_oaid_uspto_epo['oaid'])]
df_pcs_non_cleantech = df_pcs_non_cleantech.sample(n=800000, random_state=42)
# Add column 'full_oaid' with https://openalex.org/W + oaid
df_pcs_non_cleantech['full_oaid'] = 'https://openalex.org/W' + df_pcs_non_cleantech['oaid']

In [5]:
from sqlalchemy import create_engine, URL

url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    # host='134.28.58.100',
    # host='tie-workstation.tail6716.ts.net',
    host='localhost',
    port=45432,
    database='openalex_db',
)

# Create engine
engine = create_engine(url_object)
# Check if connection is successful
engine.connect()

In [13]:
# Insert temporary table into database
df_pcs_non_cleantech.to_sql('temp_table', engine, if_exists='replace', index=False)

1000

In [14]:
len(df_pcs_non_cleantech)

800000

In [15]:
# SQL query to sample df_pcs_non_cleantech column 'full_oaid' from works.id in openalex.works
sql_query = f'''
    SELECT *
    FROM openalex.works 
    JOIN temp_table ON openalex.works.id = temp_table.full_oaid
'''

In [16]:
sampled_oaids_df = pd.read_sql(sql_query, engine)

In [24]:
# Iterate over abstract_inverted_index columnc
for index, row in tqdm(sampled_oaids_df.iterrows()):
    word_index = []
    try:
        for key, value in row['abstract_inverted_index'].items():
            if key == 'InvertedIndex':
                for innerkey, innervalue in value.items():
                    for innerindex in innervalue:
                        word_index.append([innerkey, innerindex])
        # Sort list by index
        word_index.sort(key=lambda x: x[1])
        # Join first element of each list in word_index
        abstract = ' '.join([i[0] for i in word_index])
        # Add column abstract to result dataframe
        # result.loc[index, 'abstract'] = abstract
        sampled_oaids_df.at[index, 'abstract'] = abstract
        # print(result.loc[index, 'abstract'])
    except AttributeError:
        continue

0it [00:00, ?it/s]

777693it [01:08, 11343.44it/s]


In [29]:
# Drop all columns where abstract is None
sampled_oaids_df = sampled_oaids_df[sampled_oaids_df['abstract'].notna()]

In [31]:
sampled_oaids_df.to_json('/mnt/hdd01/patentsview/Non Cleantech Patents - Classifier Set/df_oaids_non_cleantech.json', orient='records')

# EPO

In [7]:
df_epo = pd.read_csv('/mnt/hdd01/PATSTAT Working Directory/PATSTAT/Cleantech Patent Raw Data/cleantech_ep_granted.csv', dtype={'publn_nr': str})

In [11]:
from sqlalchemy import create_engine, URL

url_object = URL.create(
    drivername='postgresql+psycopg2',
    username='tie',
    password='TIE%2023!tuhh',
    # host='134.28.58.100',
    # host='tie-workstation.tail6716.ts.net',
    host='localhost',
    port=25432,
    database='Patstat',
)

# Create engine
engine = create_engine(url_object)
# Check if connection is successful
engine.connect()

DETAIL:  The database was created using collation version 2.31, but the operating system provides version 2.36.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE "Patstat" REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [36]:
publn_nr_cleantech = tuple(df_epo['publn_nr'].tolist())
publn_nr_cleantech[0:10]

('1343716',
 '1276322',
 '1556493',
 '1171962',
 '1242729',
 '0128347',
 '0161004',
 '0464372',
 '0656321',
 '1227840')

In [45]:
sql_query_epo = """
    SELECT pub.*
    FROM tls211_pat_publn AS pub
    JOIN tls201_appln AS appln ON pub.appln_id = appln.appln_id
    JOIN tls224_appln_cpc AS cpc ON pub.appln_id = cpc.appln_id
    WHERE pub.publn_nr NOT IN %(publn_nr_list)s
    AND appln.appln_auth = 'EP'
    AND appln.granted = 'Y'
    AND cpc.cpc_class_symbol NOT LIKE '%%Y02%%'
    AND NOT EXISTS (
        SELECT 1
        FROM tls224_appln_cpc as cpc2
        WHERE cpc2.appln_id = pub.appln_id 
        AND cpc2.cpc_class_symbol LIKE '%%Y02%%'
    )
    ORDER BY RANDOM()
    LIMIT 200000;
"""

In [46]:
params = {'publn_nr_list': publn_nr_cleantech}
df_epo_non_cleantech = pd.read_sql(sql_query_epo, engine, params=params)
# Maybe I should also extract appln info and cpc info for each patent

In [53]:
# Drop all columns except publn_nr, appln_id and publn_date
df_epo_non_cleantech = df_epo_non_cleantech[['publn_nr', 'appln_id', 'publn_date']]